### Re-run the following reference as spark ML initial try 

- Main method 
    - CF (Collaborative Filtering)
        -  from pyspark.mllib.recommendation import ALS
            - https://spark.apache.org/docs/latest/mllib-collaborative-filtering.html

        - from pyspark.mllib.recommendation import MatrixFactorizationModel

- Credit 
    - https://github.com/jadianes/spark-movie-lens/blob/master/notebooks/building-recommender.ipynb

In [53]:
!python --version
! date

Python 3.5.5 :: Anaconda, Inc.
Sat Mar 17 08:31:01 GMT 2018


In [54]:
!conda env list

# conda environments:
#
base                     /Users/yennanliu/anaconda3
ds_dash                  /Users/yennanliu/anaconda3/envs/ds_dash
pyspark_              *  /Users/yennanliu/anaconda3/envs/pyspark_
pytorch_                 /Users/yennanliu/anaconda3/envs/pytorch_



In [3]:
# ref  : Spark context 'sc' not defined
# https://stackoverflow.com/questions/30763951/spark-context-sc-not-defined


from pyspark import SparkContext
sc =SparkContext()

In [4]:
sc

<SparkContext master=local[*] appName=pyspark-shell>

In [43]:
# download dataset

"""

1)  please download the 2 dataset manually  :

complete_dataset_url = 'http://files.grouplens.org/datasets/movielens/ml-latest.zip'
small_dataset_url = 'http://files.grouplens.org/datasets/movielens/ml-latest-small.zip'


2) and save them under :

/Users/yennanliu/movie_recommendation/datasets


3) then the dataset are here in this spark notebook demo case :

yennanliu@yennanliude-MacBook-Pro:~/movie_recommendation/datasets$ ls
ml-latest/       ml-latest-small/
yennanliu@yennanliude-MacBook-Pro:~/movie_recommendation/datasets$ 

"""




"""
### comment the original commands here ###


# download dataset


complete_dataset_url = 'http://files.grouplens.org/datasets/movielens/ml-latest.zip'
small_dataset_url = 'http://files.grouplens.org/datasets/movielens/ml-latest-small.zip'


import os

datasets_path = os.path.join('..', 'datasets')

complete_dataset_path = os.path.join(datasets_path, 'ml-latest.zip')
small_dataset_path = os.path.join(datasets_path, 'ml-latest-small.zip')

# --- fix here for python 3 --- #
# https://stackoverflow.com/questions/17960942/attributeerror-module-object-has-no-attribute-urlretrieve


import urllib.request
data = urllib.request.urlretrieve("http://...")

small_f = urllib.request.urlretrieve (small_dataset_url, small_dataset_path)
complete_f = urllib.request.urlretrieve (complete_dataset_url, complete_dataset_path)

import zipfile

with zipfile.ZipFile(small_dataset_path, "r") as z:
    z.extractall(datasets_path)

with zipfile.ZipFile(complete_dataset_path, "r") as z:
    z.extractall(datasets_path)
    
    
"""

'\n### comment the original commands here ###\n\n\n# download dataset\n\n\ncomplete_dataset_url = \'http://files.grouplens.org/datasets/movielens/ml-latest.zip\'\nsmall_dataset_url = \'http://files.grouplens.org/datasets/movielens/ml-latest-small.zip\'\n\n\nimport os\n\ndatasets_path = os.path.join(\'..\', \'datasets\')\n\ncomplete_dataset_path = os.path.join(datasets_path, \'ml-latest.zip\')\nsmall_dataset_path = os.path.join(datasets_path, \'ml-latest-small.zip\')\n\n# --- fix here for python 3 --- #\n# https://stackoverflow.com/questions/17960942/attributeerror-module-object-has-no-attribute-urlretrieve\n\n\nimport urllib.request\ndata = urllib.request.urlretrieve("http://...")\n\nsmall_f = urllib.request.urlretrieve (small_dataset_url, small_dataset_path)\ncomplete_f = urllib.request.urlretrieve (complete_dataset_url, complete_dataset_path)\n\nimport zipfile\n\nwith zipfile.ZipFile(small_dataset_path, "r") as z:\n    z.extractall(datasets_path)\n\nwith zipfile.ZipFile(complete_data

In [16]:
# --- modify here for user customize case --- # 

datasets_path = '/Users/yennanliu/movie_recommendation/datasets/'


small_ratings_file = os.path.join(datasets_path, 'ml-latest-small', 'ratings.csv')

small_ratings_raw_data = sc.textFile(small_ratings_file)
small_ratings_raw_data_header = small_ratings_raw_data.take(1)[0]

In [20]:
small_ratings_data = small_ratings_raw_data.filter(lambda line: line!=small_ratings_raw_data_header)\
    .map(lambda line: line.split(",")).map(lambda tokens: (tokens[0],tokens[1],tokens[2])).cache()
    
    

In [21]:
small_ratings_data

PythonRDD[5] at RDD at PythonRDD.scala:48

In [22]:
small_ratings_data.take(3)

[('1', '31', '2.5'), ('1', '1029', '3.0'), ('1', '1061', '3.0')]

In [23]:
small_movies_file = os.path.join(datasets_path, 'ml-latest-small', 'movies.csv')

small_movies_raw_data = sc.textFile(small_movies_file)
small_movies_raw_data_header = small_movies_raw_data.take(1)[0]

small_movies_data = small_movies_raw_data.filter(lambda line: line!=small_movies_raw_data_header)\
    .map(lambda line: line.split(",")).map(lambda tokens: (tokens[0],tokens[1])).cache()
    
small_movies_data.take(3)

[('1', 'Toy Story (1995)'),
 ('2', 'Jumanji (1995)'),
 ('3', 'Grumpier Old Men (1995)')]

In [28]:
training_RDD, validation_RDD, test_RDD = small_ratings_data.randomSplit([6, 2, 2])
validation_for_predict_RDD = validation_RDD.map(lambda x: (x[0], x[1]))
test_for_predict_RDD = test_RDD.map(lambda x: (x[0], x[1]))


In [34]:
'{} {}'.format('one', 'two')

'one two'

In [37]:
from pyspark.mllib.recommendation import ALS
import math

seed = 30
iterations = 10
regularization_parameter = 0.1
ranks = [4, 8, 12]
errors = [0, 0, 0]
err = 0
tolerance = 0.02

min_error = float('inf')
best_rank = -1
best_iteration = -1
for rank in ranks:
    model = ALS.train(training_RDD, rank, seed=seed, iterations=iterations,
                      lambda_=regularization_parameter)
    predictions = model.predictAll(validation_for_predict_RDD).map(lambda r: ((r[0], r[1]), r[2]))
    rates_and_preds = validation_RDD.map(lambda r: ((int(r[0]), int(r[1])), float(r[2]))).join(predictions)
    error = math.sqrt(rates_and_preds.map(lambda r: (r[1][0] - r[1][1])**2).mean())
    errors[err] = error
    err += 1
    # --- fix here for python 3 --- #
    print ('For rank %s the RMSE is %s' % (rank, error))
    if error < min_error:
        min_error = error
        best_rank = rank
# --- fix here for python 3 --- #
print ('The best model was trained with rank %s' % best_rank)

For rank 4 the RMSE is 0.9484775108284476
For rank 8 the RMSE is 0.9542958228942111
For rank 12 the RMSE is 0.9496177208626555
The best model was trained with rank 4


In [38]:
predictions.take(3)

[((265, 1084), 4.005518221252069),
 ((605, 1084), 2.8782891719277015),
 ((561, 1084), 3.2950294269434384)]

In [39]:
rates_and_preds.take(3)

[((563, 44191), (4.0, 4.189270123281259)),
 ((452, 8860), (2.0, 2.2496701491864757)),
 ((153, 3825), (3.0, 3.352375845806924))]

In [40]:
model = ALS.train(training_RDD, best_rank, seed=seed, iterations=iterations,
                      lambda_=regularization_parameter)
predictions = model.predictAll(test_for_predict_RDD).map(lambda r: ((r[0], r[1]), r[2]))
rates_and_preds = test_RDD.map(lambda r: ((int(r[0]), int(r[1])), float(r[2]))).join(predictions)
error = math.sqrt(rates_and_preds.map(lambda r: (r[1][0] - r[1][1])**2).mean())
    
print ('For testing data the RMSE is %s' % (error))

For testing data the RMSE is 0.9400121188663132


In [45]:
# Load the complete dataset file
complete_ratings_file = os.path.join(datasets_path, 'ml-latest', 'ratings.csv')
complete_ratings_raw_data = sc.textFile(complete_ratings_file)
complete_ratings_raw_data_header = complete_ratings_raw_data.take(1)[0]

# Parse
complete_ratings_data = complete_ratings_raw_data.filter(lambda line: line!=complete_ratings_raw_data_header)\
    .map(lambda line: line.split(",")).map(lambda tokens: (int(tokens[0]),int(tokens[1]),float(tokens[2]))).cache()

    
    
print ("There are %s recommendations in the complete dataset" % (complete_ratings_data.count()))

There are 26024289 recommendations in the complete dataset


In [48]:
training_RDD, test_RDD = complete_ratings_data.randomSplit([7, 3], seed=30)

complete_model = ALS.train(training_RDD, best_rank, seed=seed, 
                           iterations=iterations, lambda_=regularization_parameter)

In [49]:
test_for_predict_RDD = test_RDD.map(lambda x: (x[0], x[1]))

predictions = complete_model.predictAll(test_for_predict_RDD).map(lambda r: ((r[0], r[1]), r[2]))
rates_and_preds = test_RDD.map(lambda r: ((int(r[0]), int(r[1])), float(r[2]))).join(predictions)
error = math.sqrt(rates_and_preds.map(lambda r: (r[1][0] - r[1][1])**2).mean())
    
print ('For testing data the RMSE is %s' % (error))

For testing data the RMSE is 0.8312223761867484


In [50]:
complete_movies_file = os.path.join(datasets_path, 'ml-latest', 'movies.csv')
complete_movies_raw_data = sc.textFile(complete_movies_file)
complete_movies_raw_data_header = complete_movies_raw_data.take(1)[0]

# Parse
complete_movies_data = complete_movies_raw_data.filter(lambda line: line!=complete_movies_raw_data_header)\
    .map(lambda line: line.split(",")).map(lambda tokens: (int(tokens[0]),tokens[1],tokens[2])).cache()

complete_movies_titles = complete_movies_data.map(lambda x: (int(x[0]),x[1]))
    
print ("There are %s movies in the complete dataset" % (complete_movies_titles.count()))

There are 45843 movies in the complete dataset


In [51]:
def get_counts_and_averages(ID_and_ratings_tuple):
    nratings = len(ID_and_ratings_tuple[1])
    return ID_and_ratings_tuple[0], (nratings, float(sum(x for x in ID_and_ratings_tuple[1]))/nratings)

movie_ID_with_ratings_RDD = (complete_ratings_data.map(lambda x: (x[1], x[2])).groupByKey())
movie_ID_with_avg_ratings_RDD = movie_ID_with_ratings_RDD.map(get_counts_and_averages)
movie_rating_counts_RDD = movie_ID_with_avg_ratings_RDD.map(lambda x: (x[0], x[1][0]))

In [52]:
new_user_ID = 0

# The format of each line is (userID, movieID, rating)
new_user_ratings = [
     (0,260,9), # Star Wars (1977)
     (0,1,8), # Toy Story (1995)
     (0,16,7), # Casino (1995)
     (0,25,8), # Leaving Las Vegas (1995)
     (0,32,9), # Twelve Monkeys (a.k.a. 12 Monkeys) (1995)
     (0,335,4), # Flintstones, The (1994)
     (0,379,3), # Timecop (1994)
     (0,296,7), # Pulp Fiction (1994)
     (0,858,10) , # Godfather, The (1972)
     (0,50,8) # Usual Suspects, The (1995)
    ]
new_user_ratings_RDD = sc.parallelize(new_user_ratings)
print ('New user ratings: %s' % new_user_ratings_RDD.take(10))

New user ratings: [(0, 260, 9), (0, 1, 8), (0, 16, 7), (0, 25, 8), (0, 32, 9), (0, 335, 4), (0, 379, 3), (0, 296, 7), (0, 858, 10), (0, 50, 8)]


In [55]:
complete_data_with_new_ratings_RDD = complete_ratings_data.union(new_user_ratings_RDD)

In [57]:
from time import time

t0 = time()
new_ratings_model = ALS.train(complete_data_with_new_ratings_RDD, best_rank, seed=seed, 
                              iterations=iterations, lambda_=regularization_parameter)
tt = time() - t0

print ("New model trained in %s seconds" % round(tt,3))

New model trained in 186.852 seconds


In [58]:
new_user_ratings_ids = map(lambda x: x[1], new_user_ratings) # get just movie IDs
# keep just those not on the ID list (thanks Lei Li for spotting the error!)
new_user_unrated_movies_RDD = (complete_movies_data.filter(lambda x: x[0] not in new_user_ratings_ids).map(lambda x: (new_user_ID, x[0])))

# Use the input RDD, new_user_unrated_movies_RDD, with new_ratings_model.predictAll() to predict new ratings for the movies
new_user_recommendations_RDD = new_ratings_model.predictAll(new_user_unrated_movies_RDD)

In [59]:
# Transform new_user_recommendations_RDD into pairs of the form (Movie ID, Predicted Rating)
new_user_recommendations_rating_RDD = new_user_recommendations_RDD.map(lambda x: (x.product, x.rating))
new_user_recommendations_rating_title_and_count_RDD = \
    new_user_recommendations_rating_RDD.join(complete_movies_titles).join(movie_rating_counts_RDD)
new_user_recommendations_rating_title_and_count_RDD.take(3)

[(110593,
  ((6.395908451737249, "Full House (O. Henry's Full House) (1952)"), 9)),
 (104451,
  ((4.833250006473915,
    'Dealing: Or the Berkeley-to-Boston Forty-Brick Lost-Bag Blues (1972)'),
   1)),
 (174085, ((6.924064511855505, 'Fast and Furry-Ous (1949)'), 5))]

In [60]:
new_user_recommendations_rating_title_and_count_RDD = \
    new_user_recommendations_rating_title_and_count_RDD.map(lambda r: (r[1][0][1], r[1][0][0], r[1][1]))

In [62]:
top_movies = new_user_recommendations_rating_title_and_count_RDD.filter(lambda r: r[2]>=25).takeOrdered(25, key=lambda x: -x[1])

print ('TOP recommended movies (with more than 25 reviews):\n%s' %
        '\n'.join(map(str, top_movies)))

TOP recommended movies (with more than 25 reviews):
('Baseball (1994)', 9.100399853199661, 29)
('"I', 9.019731089887813, 63)
('Rabbit Fire (1951)', 8.92529408787243, 35)
('"Human Condition III', 8.81692817401003, 82)
('Duck Amuck (1953)', 8.761438822704264, 178)
('"Lonely Wife', 8.727803203868483, 31)
('Harakiri (Seppuku) (1962)', 8.717197500135654, 610)
('Death on the Staircase (Soupçons) (2004)', 8.71268283064947, 95)
('"Godfather', 8.687634559928256, 57070)
('Neighbours (1952)', 8.660087410228272, 29)
('The War (2007)', 8.653926817024384, 44)
('Connections (1978)', 8.63871972566877, 48)
('Winning Time: Reggie Miller vs. The New York Knicks (2010)', 8.62670619667903, 43)
('Dimensions of Dialogue (Moznosti dialogu) (1982)', 8.616723677287581, 51)
('Crooks in Clover (a.k.a. Monsieur Gangster) (Les tontons flingueurs) (1963)', 8.61465746889479, 46)
('"Civil War', 8.602210613372119, 400)
('Planet Earth (2006)', 8.600185461809389, 754)
('"Life of Oharu', 8.598195047107168, 44)
('"Century 

In [63]:
my_movie = sc.parallelize([(0, 500)]) # Quiz Show (1994)
individual_movie_rating_RDD = new_ratings_model.predictAll(new_user_unrated_movies_RDD)
individual_movie_rating_RDD.take(1)

[Rating(user=0, product=116688, rating=1.9833942568488254)]

In [64]:
from pyspark.mllib.recommendation import MatrixFactorizationModel

model_path = os.path.join('..', 'models', 'movie_lens_als')

# Save and load model
model.save(sc, model_path)
same_model = MatrixFactorizationModel.load(sc, model_path)

In [65]:
# end of  the re-run spark demo 